In [4]:
!pip install torch

# Build model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class PoseAdvisor(nn.Module):
    def __init__(self):
        self.model = nn.Sequential(
            nn